In [ ]:
from zipfile import ZipFile as zf
train_zip = zf('/kaggle/input/dogs-vs-cats/train.zip', 'r')
train_zip.extractall()
train_zip.close()
test_zip = zf('/kaggle/input/dogs-vs-cats/test1.zip', 'r')
test_zip.extractall()
test_zip.close()

In [ ]:
import os
os.mkdir('./train1')
os.mkdir('./train1/cats')
os.mkdir('./train1/dogs')
import shutil

for file_name in os.listdir('./train'):
    if file_name.split('.')[0] == 'cat':
        shutil.copy(os.path.join('./train/', file_name) , os.path.join('./train1/cats', file_name))
    elif file_name.split('.')[0] == 'dog':
        shutil.copy(os.path.join('./train/', file_name) , os.path.join('./train1/dogs', file_name))

In [ ]:
print(len(os.listdir('./train1/dogs')))


In [ ]:
os.mkdir('./training_set')
os.mkdir('./training_set/cats')
os.mkdir('./training_set/dogs')
os.mkdir('./val_set')
os.mkdir('./val_set/cats')
os.mkdir('./val_set/dogs')
import random
traincats = os.listdir('./train1/cats')
random.shuffle(traincats)
traindogs = os.listdir('./train1/dogs')
random.shuffle(traindogs)
for file_name in traincats[:10000]:
    shutil.copy(os.path.join('./train1/cats', file_name) , os.path.join('./training_set/cats', file_name))
for file_name in traincats[10000:]:
    shutil.copy(os.path.join('./train1/cats', file_name) , os.path.join('./val_set/cats', file_name))
for file_name in traindogs[:10000]:
    shutil.copy(os.path.join('./train1/dogs', file_name) , os.path.join('./training_set/dogs', file_name))
for file_name in traindogs[10000:]:
    shutil.copy(os.path.join('./train1/dogs', file_name) , os.path.join('./val_set/dogs', file_name))

In [ ]:
print(len(os.listdir('./training_set/cats/')))
print(len(os.listdir('./training_set/dogs/')))
print(len(os.listdir('./val_set/cats/')))
print(len(os.listdir('./val_set/dogs/')))

In [ ]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger
# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    dropout = Dropout(0.5)(class1)
    output = Dense(1, activation='sigmoid')(dropout)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['binary_accuracy'])
    # compile model
    return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['binary_accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_binary_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.show()
# run the test harness for evaluating a model
def run_test_harness():
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]  
    csv_logger = CSVLogger('training.log', separator=',', append=False)
    # prepare iterator
    train_it = datagen.flow_from_directory('./training_set',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('./val_set/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    # fit model
    history = model.fit(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1,callbacks=[csv_logger])
    
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))
    print(history.history)
    # learning curves
    summarize_diagnostics(history)
    model.save('final_model.h5')
    
    
run_test_harness()

In [ ]:
print(os.listdir('./test1')[20:30]) # print the first ten file names in the training set

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img
from matplotlib import pyplot
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.models import load_model
img_file = './test1/8920.jpg'
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img
img = load_image(img_file)
model = load_model('./final_model.h5')
result = model.predict(img)

img_classes = ['cat','dog']
print(img_classes[int(result[0][0])])

img_test = load_img(img_file, target_size=(224, 224))
arr_ = np.squeeze(img_test) # you can give axis attribute if you wanna squeeze in specific dimension
pyplot.imshow(arr_)
pyplot.show()


In [ ]:
#Confution Matrix and Classification Report
from matplotlib import pyplot
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import itertools
datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
datagen.mean = [123.68, 116.779, 103.939]  

# prepare iterator
#train_it = datagen.flow_from_directory('./training_set',class_mode='binary', batch_size=64, target_size=(224, 224))
test_it = datagen.flow_from_directory('./val_set/', batch_size=10, target_size=(224, 224),shuffle=False)
model = load_model('./final_model.h5')
#Y_pred = model.predict_generator(test_it, len(test_it) // 64+1)
predictions =model.predict(x=test_it,verbose=0)
#print(test_it.classes[10:50])
#print(predictions[0:10])
#y_pred = np.argmax(predictions, axis=1)
#print(predictions)
#print(y_pred)
print('Confusion Matrix')
cm = confusion_matrix(test_it.classes, np.round(predictions))
print(cm)
print('Classification Report')
target_names = ['Cats', 'Dogs']
print(classification_report(test_it.classes, np.round(predictions), target_names=target_names))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=pyplot.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    pyplot.imshow(cm, interpolation='nearest', cmap=cmap)
    pyplot.title(title)
    pyplot.colorbar()
    tick_marks = np.arange(len(classes))
    pyplot.xticks(tick_marks, classes, rotation=45)
    pyplot.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        pyplot.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    pyplot.tight_layout()
    pyplot.ylabel('True label')
    pyplot.xlabel('Predicted label')
    
cm_labels = ["cat","dog"]
plot_confusion_matrix(cm=cm,classes = cm_labels, title = "CM")